In [ ]:
import os
from kalshi import auth, constants, utils
from kalshi.rest import exchange, market, portfolio
from dotenv import load_dotenv
import numpy as np
import pandas as pd

constants.use_demo() # or constants.use_prod()

# Provide your credentials here
KALSHI_ACCESS_KEY = os.getenv("DEMO_KEYID")
KALSHI_PRIVATE_KEY_PATH = os.getenv("DEMO_KEYFILE")

auth.set_key(KALSHI_ACCESS_KEY, KALSHI_PRIVATE_KEY_PATH)

status = exchange.GetExchangeStatus()
status


## Events, markets, trades

In [ ]:
# Discover markets
mkts = market.GetMarkets(limit=50)
print(len(mkts.get("markets", [])), list(mkts.keys()))
print(mkts.get("markets", [])[0])


In [ ]:
# Get event information
event_ticker = "KXPRESNOMD-28"
evt = market.GetEvent(event_ticker=event_ticker)
evt

In [ ]:
market_ticker = "KXPRESNOMD-28-GN"
from time import time

# Get trades from last WEEK
trades = market.GetTrades(
    ticker=market_ticker,
    min_ts=int(time()) - 7* 24 * 3600,
    max_ts=int(time()),
    limit=1000,
)
len(trades.get("trades", [])), list(trades.keys())

In [ ]:
trades["trades"][0]

In [ ]:
trades_df = pd.DataFrame(trades["trades"])
utils.calculate_volume_stats(trades_df)

In [ ]:
# calculate volume weighted average price
utils.calculate_vwap(trades_df)

In [ ]:
utils.plot_trades(trades_df)

## Order Book

In [ ]:
# Get market order book
order_book = market.GetMarketOrderbook(ticker=market_ticker)
order_book


In [ ]:
# 
import matplotlib.pyplot as plt
import numpy as np

# Extract yes bids and no asks from the orderbook
yes_bids = order_book['orderbook'].get('yes', [])
no_bids = order_book['orderbook'].get('no', [])

# Convert to arrays for plotting
if yes_bids:
    yes_prices = [bid[0] for bid in yes_bids]
    yes_sizes = [bid[1] for bid in yes_bids]
else:
    yes_prices, yes_sizes = [], []

# Convert no bids to yes asks (100 - no_price)
if no_bids:
    no_as_yes_prices = [100 - bid[0] for bid in no_bids]
    no_as_yes_sizes = [bid[1] for bid in no_bids]
else:
    no_as_yes_prices, no_as_yes_sizes = [], []

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot yes bids (green bars going left from center)
if yes_bids:
    for price, size in zip(yes_prices, yes_sizes):
        ax.barh(price, size, height=0.8, color='green', alpha=0.7, label='Yes Bids' if price == yes_prices[0] else "")

# Plot no bids converted to yes asks (red bars going right from center)
if no_bids:
    for price, size in zip(no_as_yes_prices, no_as_yes_sizes):
        ax.barh(price, size, height=0.8, color='red', alpha=0.7, label='Yes Asks (from No)' if price == no_as_yes_prices[0] else "")

# Formatting
ax.set_xlabel('Size')
ax.set_ylabel('Price (cents)')
ax.set_title(f'Order Book for {market_ticker}')
ax.grid(True, alpha=0.3)

# Add legend
if yes_bids or no_bids:
    ax.legend()

# Set y-axis limits to show reasonable price range
if yes_prices or no_as_yes_prices:
    all_prices = yes_prices + no_as_yes_prices
    min_price = max(0, min(all_prices) - 5)
    max_price = min(100, max(all_prices) + 5)
    ax.set_ylim(min_price, max_price)

plt.tight_layout()
plt.show()

# Calculate spread
spread = None
best_yes_bid = max(yes_prices) if yes_prices else None
best_yes_ask = min(no_as_yes_prices) if no_as_yes_prices else None

if best_yes_bid is not None and best_yes_ask is not None:
    spread = best_yes_ask - best_yes_bid

# Print summary
print(f"\nOrder Book Summary for {market_ticker}:")
print(f"Yes Bids: {len(yes_bids)} orders")
print(f"No Bids (as Yes Asks): {len(no_bids)} orders")
if best_yes_bid is not None:
    print(f"Best Yes Bid: {best_yes_bid}¢")
if best_yes_ask is not None:
    print(f"Best Yes Ask: {best_yes_ask}¢")
if spread is not None:
    print(f"Spread: {spread}¢")


## Portfolio and trades

In [ ]:
# Portfolio endpoints: Balance, Positions, Fills, Settlements
bal = portfolio.GetBalance()
pos = portfolio.GetPositions(limit=50)
fil = portfolio.GetFills(limit=20)
setl = portfolio.GetPortfolioSettlements(limit=20)

{"balance": bal, "positions_keys": list(pos.keys()), "fills_keys": list(fil.keys()), "settlements_keys": list(setl.keys())}


In [ ]:
import uuid

if constants.BASE_URL == constants.PROD_BASE_URL:
    raise Exception("DANGER: USING PROD")

# Place a limit buy for 1 contract at 20c Yes
order_response = portfolio.CreateOrder(
    ticker="KXPRESNOMD-28-GN",
    client_order_id=str(uuid.uuid4()),
    type="limit",      # "limit" or "market"
    action="buy",      # "buy" or "sell"
    side="yes",        # "yes" or "no"
    count=1,
    yes_price=20       # price in cents (0-100). Use 'no_price' instead if side="no"
)

order_response